In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from transformers import RobertaTokenizer, TFRobertaModel
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# 加载数据并预处理
df = pd.read_csv('data.csv')
df.dropna(inplace=True)  # 删除空值
df['label'] = LabelEncoder().fit_transform(df['label'])  # 对标签进行编码

# 使用 RoBERTa 分词器
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta_model = TFRobertaModel.from_pretrained("roberta-base")

# 数据编码函数
def encode_data(df, tokenizer, max_length=64):
    encoded = tokenizer(
        df['phrase'].tolist(),
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors="tf"
    )
    return encoded, df['label'].values

# 编码数据
encoded, y = encode_data(df, tokenizer)

# 提取 input_ids 和 attention_mask 并转换为 NumPy 数组
input_ids = encoded['input_ids'].numpy()
attention_mask = encoded['attention_mask'].numpy()

# 划分数据集为训练集和测试集
train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_y, test_y = train_test_split(
    input_ids, attention_mask, y, test_size=0.2, random_state=42)

# 创建训练数据集和测试数据集
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': train_input_ids, 'attention_mask': train_attention_mask}, train_y)).batch(128)
test_dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': test_input_ids, 'attention_mask': test_attention_mask}, test_y)).batch(128)

# 定义类别数量和模型输入层
num_classes = 3
input_ids = Input(shape=(64,), dtype="int32", name='input_ids')
attention_mask = Input(shape=(64,), dtype="int32", name='attention_mask')

# 获取 RoBERTa 模型的输出
bert_outputs = roberta_model([input_ids, attention_mask])
bert_pooled_output = bert_outputs[1]  # 使用池化输出

# 添加输出层并创建模型
outputs = Dense(units=num_classes, activation="softmax")(bert_pooled_output)
model = Model(inputs=[input_ids, attention_mask], outputs=outputs)

# 编译模型
adam = Adam(learning_rate=2e-5, epsilon=1e-08)
model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"], optimizer=adam)

# 训练模型
model.fit(train_dataset, epochs=15)

# 保存模型
model.save('my_roberta_model.h5')

# 评估模型
results = model.evaluate(test_dataset)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

# 预测和计算 F1-score
predictions = model.predict(test_dataset)
predicted_labels = np.argmax(predictions, axis=1)
f1 = f1_score(test_y, predicted_labels, average='macro')
print(f"F1 Score: {f1}")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Epoch 1/3
382/382 [==============================] - 132s 327ms/step - loss: 0.6741 - accuracy: 0.7140
Epoch 2/3
382/382 [==============================] - 125s 328ms/step - loss: 0.4180 - accuracy: 0.8329
Epoch 3/3
96/96 [==============================] - 12s 114ms/step - loss: 0.2826 - accuracy: 0.8989
Test Loss: 0.2825864553451538, Test Accuracy: 0.8988874554634094
96/96 [==============================] - 12s 111ms/step
F1 Score: 0.8980906188767069
